In [ ]:
import numpy as np
import random
import time
from Digital_twin import DigitalTwin

class InvertedPendulumGA:
    def __init__(self, population_size, num_actions, simulation_duration, action_resolution, simulation_delta_t):
        self.digital_twin = DigitalTwin()
        self.population_size = population_size
        self.parent_pool_size = 2 #parent_pool_size
        self.num_actions = num_actions
        self.simulation_duration = simulation_duration
        self.action_resolution = action_resolution
        self.simulation_delta_t = simulation_delta_t
        self.simulation_steps = simulation_duration/simulation_delta_t
        self.num_steps = int(simulation_duration / action_resolution)
        self.step_resolution = int(action_resolution / simulation_delta_t)
        self.population = [self.create_individual() for _ in range(population_size)]
        
        fitness_scores = self.evaluate_population()
        print(fitness_scores, "at start")

    def create_individual(self):
        """Create an individual sequence of actions with balanced left and right actions and boundary constraints."""
        actions = np.zeros(self.num_steps, dtype=int)  # Start with neutral actions
        # Initialize a variable to track the net movement direction and magnitude
        net_movement = 0  # Positive for right, negative for left
        
        for i in range(self.num_steps):
            if abs(net_movement) < 100:
                # If net movement is within acceptable bounds, choose any action
                action = np.random.randint(1, self.num_actions)
                # Update net movement based on the chosen action
                if action in [1, 2, 3, 4]:  # Left actions
                    net_movement -= self.digital_twin.action_map[action][1]
                else:  # Right actions
                    net_movement += self.digital_twin.action_map[action-4][1]
            elif net_movement >= 100:
                # If net movement is too far right, choose a left action to balance
                action = np.random.choice([1, 2, 3, 4])
                net_movement -= self.digital_twin.action_map[action][1]
            else:  # net_movement <= -150
                # If net movement is too far left, choose a right action to balance
                action = np.random.choice([5, 6, 7, 8])
                net_movement += self.digital_twin.action_map[action-4][1]

            actions[i] = action
        
        #print(actions)
        return actions


    def simulate(self, actions):
        """Simulate the inverted pendulum with the given actions and return a fitness score."""
        self.digital_twin.theta = 0.
        self.digital_twin.theta_dot = 0.
        self.digital_twin.x_pivot = 0.
        self.digital_twin.steps = 0.
        max_score = 0.

        action_list = actions.tolist()
        while self.digital_twin.steps < self.simulation_steps:
            if self.digital_twin.steps%self.step_resolution == 0 and len(action_list) > 0:
                action = action_list.pop(0)
                direction, duration = self.digital_twin.action_map[action]
                self.digital_twin.perform_action(direction, duration)
            theta, theta_dot, x_pivot, _ = self.digital_twin.step()
            if abs(theta) > max_score:
                max_score = abs(theta)
            if abs(self.digital_twin.x_pivot) > 99:
                #print('not good')
                return -100
        return max_score

    def evaluate_population(self):
        """Evaluate the fitness of the entire population."""
        fitness_scores = [self.simulate(individual) for individual in self.population]
        return fitness_scores

    def select_parents(self, fitness_scores):
        """Select a pool of parent individuals based on their fitness scores."""
        pool_size = min(self.parent_pool_size, len(fitness_scores))
        # Select indices of the top performers to form the pool
        top_performers_indices = np.argsort(fitness_scores)[-pool_size:]
        return [self.population[i] for i in top_performers_indices]


    def crossover(self, parent1, parent2):
        """Perform crossover between two parents to produce an offspring."""
        crossover_point = random.randint(1, self.num_steps - 1)
        offspring = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
        return offspring

    def mutate(self, individual, mutation_rate=0.2):
        """Mutate an individual's actions with a given mutation rate."""
        for i in range(self.num_steps):
            if random.random() < mutation_rate:
                individual[i] = random.randint(0, self.num_actions - 1)
        return individual

    def run_generation(self):
        """Run a single generation of the genetic algorithm, using all parents in the pool to create offspring."""
        fitness_scores = self.evaluate_population()
        parents_pool = self.select_parents(fitness_scores)
        
        # Shuffle the parents pool to randomize pairings
        np.random.shuffle(parents_pool)
        
        new_population = []
        while len(new_population) < self.population_size:
            for i in range(0, len(parents_pool), 2):
                # Break the loop if the new population is already filled
                if len(new_population) >= self.population_size:
                    break
                
                # Ensure there's a pair to process
                if i + 1 < len(parents_pool):
                    parent1 = parents_pool[i]
                    parent2 = parents_pool[i + 1]
                    offspring1 = self.crossover(parent1, parent2)
                    offspring2 = self.crossover(parent2, parent1)  # Optional: create a second offspring by reversing the parents
                    
                    # Mutate and add the new offspring to the new population
                    new_population.append(self.mutate(offspring1))
                    if len(new_population) < self.population_size:
                        new_population.append(self.mutate(offspring2))
                    
                    # If the end of the parent pool is reached but more offspring are needed, reshuffle and continue
                    if i + 2 >= len(parents_pool) and len(new_population) < self.population_size:
                        np.random.shuffle(parents_pool)

        # Replace the old population with the new one
        self.population = new_population[:self.population_size]

    def optimize(self, num_generations, fitness_threshold):
        """Optimize the inverted pendulum control over a number of generations or until an individual meets the fitness threshold."""
        for i in range(num_generations):
            self.run_generation()
            # Evaluate the population after this generation
            fitness_scores = self.evaluate_population()
            best_index = np.argmax(fitness_scores)
            best_fitness = fitness_scores[best_index]
            
            print(f"Generation: {i}, Best Fitness: {best_fitness}")
            
            # Check if the best individual meets the fitness threshold
            if best_fitness >= fitness_threshold:
                print(f"Stopping early: Individual found with fitness {best_fitness} meeting the threshold at generation {i}.")
                return self.population[best_index]
        
        # If the loop completes without returning, no individual met the threshold; return the best found
        print(f"No individual met the fitness threshold. Best fitness after {num_generations} generations is {best_fitness}.")
        return self.population[best_index]


# Example usage
ga = InvertedPendulumGA(population_size=40, num_actions=9, simulation_duration=2, action_resolution=0.2, simulation_delta_t=0.005)
best_solution = ga.optimize(num_generations=100, fitness_threshold=np.pi)

print("Best Solution:", best_solution)



---

### 3.1. Discuss the following with your group before you start: What are the specific challenges in using genetic algorithms for controlling a pendulum and achieving an upright position?

there are several problems that we could face in this scenario that are as below:

-  Nonlinear dynamics: The pendulum system is nonlinear and unstable near the upright position. Small changes in actions can lead to drastically different results.
-  Delayed effect: Each control action might have a delayed impact, making it hard to evaluate how good a sequence is early in the simulation.
-  Fitness evaluation cost: Simulating each candidate sequence with the Digital Twin is computationally expensive.


### 3.2. First set the self.delta_t in the Digital_Twin.py to 0.005 as this is what we use during the optimization process: self.delta t = 0.005.

our sample time delta_t is = 025 s and we added that in all part of our simulations 




### 3.4. How should the parameters of the genetic algorithm (e.g., population size, mutation rate, crossover rate) be determined? What strategies can be implemented to fine-tune these parameters effectively?

now lets discuss question 3.4 and go step by step
fist of all, population size:

A larger population size increases the genetic diversity among individuals in each generation, which can lead to better exploration of the solution space. For this reason, we selected a population size of 60. However, this comes at the cost of increased computational time. In an ideal scenario, even a population size as small as 4 was able to achieve a partial solution when the generation size exceeded 4. However, the result was limited—it only reached 180 degrees and was unable to maintain stability at that position.

Mutation rate:

Mutation introduces random and unpredictable variations into the population, allowing the algorithm to explore areas of the solution space that may not be reachable through crossover alone. If the mutation rate is too low, the algorithm may converge prematurely and become trapped in local optima. Conversely, if the mutation rate is too high, the search process becomes overly random, which can prevent convergence to a viable solution. In our implementation, we set the mutation rate to 10%.

Cross over rate:

The crossover rate determines how frequently two parent sequences combine to produce offspring. Common values typically range between 80% and 90%, as higher crossover rates facilitate the exchange of beneficial traits between solutions, promoting convergence. In our experiments, we began with short crossover intervals (e.g., every 2 steps) and gradually tested longer intervals to assess their impact on performance.

### 3.5.2 Can you come-up with another method of evaluation that will improve the learning rate?

Instead of relying solely on the maximum angle (theta) reached, we reward the robot based on the duration it remains balanced and centered. Additionally, a penalty is applied for any deviation in angle, encouraging more stable and controlled behavior.

### 3.5.3 Bonus question: What is fundamentally the problem with the current crossover function in the GA? Can you come up with a better version?

Instead of using one-point crossover, we can employ uniform crossover, which offers several advantages. One-point crossover is computationally simple and efficient, but it limits genetic diversity by mixing genetic material at only a single location. This often results in offspring that closely resemble one parent, potentially slowing the learning process and increasing the risk of convergence to local optima.

In contrast, uniform crossover enhances genetic diversity by randomly selecting each gene (or action) from either parent, thereby promoting broader exploration of the solution space and often leading to faster learning. However, this approach may occasionally disrupt beneficial sequences of genes inherited from parents, which could result in less stable behavior in offspring.

Overall, while one-point crossover provides more consistency, uniform crossover generally yields better performance due to its greater variability and flexibility.